# **Inicio del AB FINAL**

## **Preparacion de los datos**

En este caso, se ha decidido crear este dataframe unificando todos los CSV en un único DataFrame y añadir una columna que identifique el origen (por ejemplo, source_id). Ya que la otra es crear un diccionario con cada csv separado, pero la custion es que las estructuras de columnas son muy parecidas y no paraece que se vaya a necesitar tratamientos específicos por fichero, si no mas bien en conjunto.

Por ello, se considera que es mas optima esta opción de unificar todos los datos en el mismo Dataframe

### Ruta de los *csv*

In [ ]:
# Instalar dependencias necesarias
import subprocess
import sys

packages = [
    "pandas",
    "numpy", 
    "openpyxl",
    "matplotlib",
    "seaborn",
    "scikit-learn",
    "scipy",
    "cmdstanpy",
    "pystan",
    "prophet"
]

for package in packages:
    try:
        __import__(package.replace("-", "_"))
        print(f"✓ {package} ya está instalado")
    except ImportError:
        print(f"Instalando {package}...")
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
            print(f"✓ {package} instalado correctamente")
        except subprocess.CalledProcessError:
            print(f"⚠️ Error instalando {package}, continuando...")